# Goal: Create a tool that will find relevant data and display guide star magnitude and an SED for every object in a Magellan-formatted catalog


## Imports

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['font.family'] = "Times New Roman"
plt.rcParams['legend.loc'] = "Upper Right"
plt.rcParams['figure.dpi'] = 900

import scipy.stats as st
import seaborn as sb


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from astropy.io import ascii
from astroplan import (AltitudeConstraint, AirmassConstraint, AtNightConstraint)
from astroplan import FixedTarget, Observer
from astroplan import is_observable, is_always_observable, months_observable
from astroplan import observability_table
from astropy import constants as const
from astropy.time import Time
from astropy import units as u
from astropy.coordinates import *
from astropy.modeling.blackbody import blackbody_lambda
from astropy.table import Table
from astroquery.simbad import Simbad

import scipy.optimize as optimization
from scipy.optimize import curve_fit
from scipy.stats import linregress
from sklearn.neighbors import KernelDensity
from subprocess import *

## Custom Simbad

In [ ]:
customSimbad = Simbad()
customSimbad.add_votable_fields('sptype')

## Stars

In [ ]:
target_list = ['Vega', 'Sirius', 'HD 168607', 'HD 168625']
guide_stars = ['HD 200466B', 'HD 207966']

stars = guide_stars + target_list
constraint_stars = stars

In [ ]:
stars

In [ ]:
lco = Observer.at_site('LCO')

## Spectral type CSVs get to temp, absolute mag, and solar luminosities

In [ ]:
main_key = pd.read_csv('mainsequence.csv')
giantsiii_key = pd.read_csv('giantsiii.csv')
supergiantsi_key = pd.read_csv('supergiantsi.csv')

# Setting indexer as spectral type
main_key.set_index('SP_TYPE', inplace=True)
giantsiii_key.set_index('SP_TYPE', inplace=True)
supergiantsi_key.set_index('SP_TYPE', inplace=True)

## Spectral Type

In [ ]:
columns = {'MAIN_ID': [0], 'SP_TYPE': [0]}
vis = pd.DataFrame(data = columns)

x = 0
for i in stars:
    obj = customSimbad.query_object(stars[x])
    obj.keep_columns(['MAIN_ID', 'SP_TYPE'])
    obj = obj.to_pandas()
    vis = vis.append(obj)
    x = x + 1
    
    
vis = vis.iloc[1:]

vis = vis.values.tolist()

print(vis[3][1])

In [ ]:
x = 0
for i in stars:
    vis[x][0] = vis[x][0].decode("utf-8")
    vis[x][1] = vis[x][1].decode("utf-8")
    x = x + 1
    

vis

## Temp (k), Absolute Magnitude, Solar Luminosities

In [ ]:
x = 0
for i in stars:
    sp = vis[x][1][0:2]
    app = main_key.loc[sp]
    app = app.tolist()
    print(app)
    vis[x].extend(app)
    x = x + 1
    

vis

### The list is now [name, spectral type, temp in kelvin,  absolute magnitude, solar luminosities]

## SED

In [ ]:
# Courtesy of gist.github.com/mfouesneau, an astrophd dude in Germany

try:  # python 3
    from io import BytesIO
    from http.client import HTTPConnection
except ImportError:  # python 2
    from StringIO import StringIO as BytesIO
    from httplib import HTTPConnection

from astropy.table import Table


def query_sed(pos, radius=5):
    """ Query VizieR Photometry 
    The VizieR photometry tool extracts photometry points around a given position 
    or object name from photometry-enabled catalogs in VizieR.
    
    The VizieR photometry tool is developed by Anne-Camille Simon and Thomas Boch
    .. url:: http://vizier.u-strasbg.fr/vizier/sed/doc/
    
    Parameters
    ----------
    pos: tuple or str
        position tuple or object name
    radius: float
        position matching in arseconds.
    
    Returns
    -------
    table: astropy.Table
        VO table returned by the Vizier service.
        
    >>> query_sed((1.286804, 67.840))
    >>> query_sed("HD1")
    """
    try:
        ra, dec = pos
        target = "{0:f},{1:f}".format(ra, dec)
    except:
        target = pos
    
    url = "http:///viz-bin/sed?-c={target:s}&-c.rs={radius:f}"
    host = "vizier.u-strasbg.fr"
    port = 80
    path = "/viz-bin/sed?-c={target:s}&-c.rs={radius:f}".format(target=target, radius=radius)
    connection = HTTPConnection(host, port)
    connection.request("GET", path)
    response = connection.getresponse()
   
    table = Table.read(BytesIO(response.read()), format="votable")
    return table

In [ ]:
x = 0

for i in stars:
    s = query_sed(vis[x][0].replace(" ", ""))
    del s['_ID'], s['_tabname'], s['_RAJ2000'], s['_DEJ2000']
    s = s.to_pandas()
    #wavelength micrometer = speed of light / frequency in hz
    s['wavelength'] = (299792458/(s['sed_freq']*(1e+9)))*(1e+6)
    #sed_freq sed_flux sed_eflux sed_filter
    #GHz Jy Jy
    #float64 float32 float32 bytes32
    #1 Jy = 10-23 erg s-1 cm-2 Hz-1 = 10-26 Watts m-2 Hz-1 = FsubV
    print(vis[x])
    a = plt.scatter(np.log10(s['wavelength']), np.log10(s['sed_flux']), s=5, color = 'r')
    plt.xlim(-1, 4)
    plt.xlabel('Wavelength (log(λ/μm))')
    plt.ylabel('Flux in Janskys')
    
    labels, locations = plt.xticks()
    plt.xticks(labels, 10**labels)

    
    #plt.legend(shadow=True, loc='center left', bbox_to_anchor=(1.1, 1))
    plt.title('Spectral Energy Distribution of: ' + constraint_stars[x])

    plt.savefig(constraint_stars[x]+'_sed.png', bbox_inches='tight', pad_inches=0.25)
    plt.clf()
    plt.rcParams['figure.dpi'] = 900
    
    x = x+1

## Other Relevant Data — Table of Visibilty

In [ ]:
time_range = Time(["2019-07-18 21:00", "2019-07-19 13:00"])

In [ ]:
x = 0
for i in constraint_stars:
    constraint_stars[x] = FixedTarget.from_name(constraint_stars[x])
    x = x + 1

In [ ]:
constraint_stars

In [ ]:
#constraints = [AltitudeConstraint(10*u.deg, 80*u.deg),
#               AirmassConstraint(5), AtNightConstraint.twilight_civil()]

#table = observability_table(constraints, lco, constraint_stars, time_range = time_range)
#print(table)

In [ ]:
target_table = Table.read('targets.txt', format='ascii')
targets = [FixedTarget(coord=SkyCoord(ra=ra*u.deg, dec=dec*u.deg), name='Vega')
           for name, ra, dec in target_table]

constraints = [AltitudeConstraint(10*u.deg, 80*u.deg),
               AirmassConstraint(5), AtNightConstraint.twilight_civil()]

In [ ]:
# Are targets *ever* observable in the time range?
ever_observable = is_observable(constraints, lco, targets, time_range=time_range)

# Are targets *always* observable in the time range?
always_observable = is_always_observable(constraints, lco, targets, time_range=time_range)

# During what months are the targets ever observable?
best_months = months_observable(constraints, lco, targets)

In [ ]:
table = observability_table(constraints, lco, constraint_stars, time_range=time_range)
print(table)

table = table.to_pandas()

np.savetxt('visibility.txt', table, fmt="%-30s", header = 'Target name                  ever observable                always observable              fraction of time observable')

In [ ]:
table